<a href="https://colab.research.google.com/github/sejalkailashyadav/Notes/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔒 Ultimate Security Toolkit PRO 3.0

In [42]:
from pyngrok import ngrok

# Kill all tunnels
ngrok.kill()
print("All ngrok tunnels have been stopped.")


All ngrok tunnels have been stopped.


In [43]:
# ----------------- Install Dependencies -----------------
!pip install streamlit pyngrok cryptography pandas plotly --quiet

# ----------------- Create Enhanced App -----------------
pro_ui_app = '''
import streamlit as st
import pandas as pd, random, string, base64
from cryptography.fernet import Fernet
import plotly.express as px
import streamlit.components.v1 as components
import os

st.set_page_config(page_title="🔒 Ultimate Security Toolkit PRO 3.0", layout="wide")
st.title("🔒 Ultimate Security Toolkit PRO 3.0")

# ----------------- SESSION STATE -----------------
for key in ["password_history", "usernames", "notes", "theme"]:
    if key not in st.session_state:
        st.session_state[key] = [] if "history" in key or key in ["usernames","notes"] else "Light"

# ----------------- THEME TOGGLE -----------------
theme = st.sidebar.radio("Theme", ["Light","Dark"], index=0 if st.session_state["theme"]=="Light" else 1)
st.session_state["theme"] = theme
if theme=="Dark":
    st.markdown("<style>body{background-color:#111;color:white;}</style>", unsafe_allow_html=True)
else:
    st.markdown("<style>body{background-color:white;color:black;}</style>", unsafe_allow_html=True)

# ----------------- MODULE TABS -----------------
tab1, tab2, tab3, tab4, tab5, tab6 = st.tabs([
    "Password Generator","Password Checker","Username Generator","Encrypt/Decrypt","Secure Notes","Analytics"
])

# ----------------- PASSWORD GENERATOR -----------------
with tab1:
    st.header("🔑 Password Generator")
    length = st.slider("Password Length",8,32,16)
    upper = st.checkbox("Include Uppercase",True)
    numbers = st.checkbox("Include Numbers",True)
    symbols = st.checkbox("Include Symbols",True)
    batch = st.number_input("Number of Passwords",1,10,1)

    def gen_pwd():
        chars = string.ascii_lowercase
        if upper: chars+=string.ascii_uppercase
        if numbers: chars+=string.digits
        if symbols: chars+="!@#$%^&*()-_=+[]{}|;:,.<>?"
        return ''.join(random.choice(chars) for _ in range(length))

    if st.button("Generate Passwords"):
        passwords = [gen_pwd() for _ in range(batch)]
        st.session_state["password_history"].extend(passwords)
        df = pd.DataFrame({"Passwords": passwords})
        st.dataframe(df)
        # Copy buttons
        for p in passwords:
            html_code = f"""
            <div style="display:flex; align-items:center; margin-bottom:5px;">
                <input value="{p}" id="copy_{p}" readonly style="width:300px; padding:5px;"/>
                <button onclick="navigator.clipboard.writeText(document.getElementById('copy_{p}').value)">Copy</button>
            </div>
            """
            components.html(html_code, height=40)
        # Export CSV
        df_all = pd.DataFrame({"Passwords": st.session_state["password_history"]})
        st.download_button("📥 Export All Passwords", df_all.to_csv(index=False).encode(), "passwords.csv")

# ----------------- PASSWORD CHECKER -----------------
with tab2:
    st.header("🛡 Password Strength Checker")
    pwd = st.text_input("Enter password to check")
    if pwd:
        score = 0
        issues=[]
        if any(c.islower() for c in pwd): score+=1
        else: issues.append("Add lowercase")
        if any(c.isupper() for c in pwd): score+=1
        else: issues.append("Add uppercase")
        if any(c.isdigit() for c in pwd): score+=1
        else: issues.append("Add numbers")
        if any(c in "!@#$%^&*()-_=+[]{}|;:,.<>?" for c in pwd): score+=1
        else: issues.append("Add symbols")
        st.info(f"Strength Score: {score}/4")
        if score<3: st.warning(f"Weak password! Issues: {', '.join(issues)}")
        else: st.success("Strong password!")

# ----------------- USERNAME GENERATOR -----------------
with tab3:
    st.header("👤 Username Generator")
    prefix = st.text_input("Prefix")
    suffix = st.text_input("Suffix")
    u_length = st.slider("Random Part Length",4,12,6)
    u_batch = st.number_input("Number of usernames",1,10,1)
    if st.button("Generate Usernames"):
        usernames = [prefix + ''.join(random.choices(string.ascii_letters+string.digits,k=u_length)) + suffix for _ in range(u_batch)]
        st.session_state["usernames"].extend(usernames)
        df = pd.DataFrame({"Usernames": usernames})
        st.dataframe(df)
        # Copy buttons
        for u in usernames:
            html_code = f"""
            <div style="display:flex; align-items:center; margin-bottom:5px;">
                <input value="{u}" id="copy_{u}" readonly style="width:300px; padding:5px;"/>
                <button onclick="navigator.clipboard.writeText(document.getElementById('copy_{u}').value)">Copy</button>
            </div>
            """
            components.html(html_code, height=40)
        # Export CSV
        df_all = pd.DataFrame({"Usernames": st.session_state["usernames"]})
        st.download_button("📥 Export All Usernames", df_all.to_csv(index=False).encode(), "usernames.csv")

# ----------------- ENCRYPT/DECRYPT -----------------
with tab4:
    st.header("🔐 AES Encrypt/Decrypt")
    key_input = st.text_input("Secret Key")
    text = st.text_area("Text")
    action = st.radio("Action", ["Encrypt","Decrypt"])
    if st.button("Run Encryption/Decryption"):
        if not key_input or not text: st.error("Key & text required")
        else:
            key = base64.urlsafe_b64encode(key_input.ljust(32)[:32].encode())
            f = Fernet(key)
            try:
                if action=="Encrypt":
                    enc=f.encrypt(text.encode()).decode()
                    st.success(enc)
                    components.html(f'<button onclick="navigator.clipboard.writeText(`{enc}`)">Copy Encrypted</button>', height=30)
                else:
                    dec=f.decrypt(text.encode()).decode()
                    st.success(dec)
                    components.html(f'<button onclick="navigator.clipboard.writeText(`{dec}`)">Copy Decrypted</button>', height=30)
            except Exception as e:
                st.error(e)

# ----------------- SECURE NOTES -----------------
with tab5:
    st.header("📝 Secure Notes Vault")
    title = st.text_input("Title")
    note = st.text_area("Note content")
    if st.button("Add Note"):
        if note.strip(): st.session_state["notes"].append({"Title":title.strip(), "Note":note.strip()})
    if st.session_state["notes"]:
        df = pd.DataFrame(st.session_state["notes"])
        search = st.text_input("Search Notes")
        filtered = df[df.apply(lambda row: search.lower() in row.to_string().lower(), axis=1)]
        st.dataframe(filtered)
        st.download_button("📥 Export Notes CSV", df.to_csv(index=False).encode(), "notes.csv")
        uploaded_file = st.file_uploader("📤 Import Notes CSV", type="csv")
        if uploaded_file:
            df_import = pd.read_csv(uploaded_file)
            st.session_state["notes"].extend(df_import.to_dict('records'))
            st.success("Imported successfully!")

# ----------------- ANALYTICS -----------------
with tab6:
    st.header("📊 Analytics Dashboard")
    st.metric("Passwords Generated", len(st.session_state["password_history"]))
    st.metric("Usernames Generated", len(st.session_state["usernames"]))
    st.metric("Notes Added", len(st.session_state["notes"]))

    if st.session_state["password_history"]:
        df_plot = pd.DataFrame({"Passwords": st.session_state["password_history"]})
        df_plot["Length"] = df_plot["Passwords"].apply(len)
        fig = px.histogram(df_plot, x="Length", nbins=10, title="Password Length Distribution")
        st.plotly_chart(fig, use_container_width=True)
'''

with open("app.py","w") as f:
    f.write(pro_ui_app)

# ----------------- NGROK -----------------
from pyngrok import ngrok
ngrok.set_auth_token("341MFC0ROB3trf1NOV6lZilLbpb_37K75BTLd6uwvDZ2hdEBF")  # Replace with your ngrok token

import threading, os
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501 --server.headless true")
threading.Thread(target=run_streamlit).start()

public_url = ngrok.connect(8501, proto="http")
print("Your Ultimate Security Toolkit PRO 3.0 is live at:", public_url)


Your Ultimate Security Toolkit PRO 3.0 is live at: NgrokTunnel: "https://noncontrollable-jerrell-courtly.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')